In [1]:
'''
Dandan Wei
Jan 19, 2022

This script crop the Landsat EVI and QA_pixel into the NYC shape, using the Landsat projection.
Input dataset:
        Landsat7/8 SR_EVI tiff
        Landsat7/8 QA_Pixel tiff
Output dataset:
        Cropped Landsat7/8 SR_EVI tiff
        Cropped Landsat7/8 QA_Pixel tiff

'''

import gdal, ogr, osr
import matplotlib.pyplot as plt
import numpy as np
import os
import rasterio
import math
import pandas as pd
from datetime import datetime
import glob

################################################################
#                                                              #
#               Define directories and filenames               #
#                                                              #
################################################################
DataDir     = "/data0/dwei/Landsat/2021/extracted/" 
CropDir     = "/data0/dwei/Landsat/2021/cropped_to_NYC/" 
FigDir_temp = "/data0/dwei/temp/"
ShpeDir     = "/data0/dwei/NYCshapefiles/"
NYCshapefile= 'geo_export_a8b5d138-820a-4931-8d11-fe8604ea5bd1.shp'

################################################################
#                                                              #
#               Crop the Landsat data into NYC                 #
#                                                              #
################################################################
def get_NYC_bbox(shapefile, target_EPSG):
    ''' 
    To get the bounding box of NYC
    '''   
    # define shapefile driver
    drv         = ogr.GetDriverByName('ESRI Shapefile') 
    ds_in       = drv.Open(shapefile,0) 
    lyr_in      = ds_in.GetLayer()            # grab layer
    sourceprj   = lyr_in.GetSpatialRef()      # source projection
    targetprj   = osr.SpatialReference()
    targetprj.ImportFromEPSG(target_EPSG)     # target projection
    transform   = osr.CoordinateTransformation(sourceprj, targetprj)

    shp         = lyr_in.GetExtent()
    ll_x,ll_y,_ = transform.TransformPoint(shp[0],shp[2]) # reprojected bounds for city
    ur_x,ur_y,_ = transform.TransformPoint(shp[1],shp[3]) # reprojected bounds for city
    bbox        = [ll_x, ll_y, ur_x, ur_y]    # bounding box
    # print("The original prj is :\n", sourceprj)
    # print("The target   prj is :\n", targetprj)
    print('NYC bbox :', bbox)
    return bbox  

def get_geotransform(raster_file_path):
    ''' 
    To print the GeoTransform of a raster file
    '''
    ds = gdal.Open(raster_file_path)
    gt = ds.GetGeoTransform()
    print(gt)
    return gt 

# Parameters  
target_xRes = 30.0
target_yRes = 30.0
target_EPSG = 32618    # Landsat projection
bbox = get_NYC_bbox(ShpeDir+NYCshapefile, target_EPSG) # NYC minx miny maxx maxy

EVI_files = glob.glob(DataDir+'*SR_EVI*')
QA_files = glob.glob(DataDir+'*QA_PIXEL*') 
Band_files = glob.glob(DataDir+'*SR_B*') 
# print(np.size(QA_files))
# print(np.size(QA_files))
print(np.size(Band_files))

# Crop the Landsat EVI raster files
for file in EVI_files:
    filename = file[len(DataDir):]
    output_evi = CropDir + 'NYC_' + filename
#     print(output_evi)
    gdal.Warp(output_evi, \
          file, \
          dstSRS = "EPSG:32618", \
          xRes = target_xRes, yRes = target_yRes, \
          outputBounds=bbox)  # outputBounds (minX, minY, maxX, maxY)
    get_geotransform(output_evi)  
    
# Crop the Landsat QA_pixel raster files
for file in QA_files:
    filename = file[len(DataDir):]
    output   = CropDir + 'NYC_' + filename
#     print(output)
    gdal.Warp(output, \
          file, \
          dstSRS = "EPSG:32618", \
          xRes = target_xRes, yRes = target_yRes, \
          outputBounds=bbox)  # outputBounds (minX, minY, maxX, maxY)
    get_geotransform(output)

# Crop the Landsat 7/8 band files
for file in Band_files:
    filename = file[len(DataDir):]
    output   = CropDir + 'NYC_' + filename
#     print(output)
    gdal.Warp(output, \
          file, \
          dstSRS = "EPSG:32618", \
          xRes = target_xRes, yRes = target_yRes, \
          outputBounds=bbox)  # outputBounds (minX, minY, maxX, maxY)
    get_geotransform(output)
    
    

NYC bbox : [563086.4981809029, 4483346.578300821, 609468.3374729961, 4530140.279354534]
74
(563086.4981809029, 30.0, 0.0, 4530140.279354534, 0.0, -30.0)
(563086.4981809029, 30.0, 0.0, 4530140.279354534, 0.0, -30.0)
(563086.4981809029, 30.0, 0.0, 4530140.279354534, 0.0, -30.0)
(563086.4981809029, 30.0, 0.0, 4530140.279354534, 0.0, -30.0)
(563086.4981809029, 30.0, 0.0, 4530140.279354534, 0.0, -30.0)
(563086.4981809029, 30.0, 0.0, 4530140.279354534, 0.0, -30.0)
(563086.4981809029, 30.0, 0.0, 4530140.279354534, 0.0, -30.0)
(563086.4981809029, 30.0, 0.0, 4530140.279354534, 0.0, -30.0)
(563086.4981809029, 30.0, 0.0, 4530140.279354534, 0.0, -30.0)
(563086.4981809029, 30.0, 0.0, 4530140.279354534, 0.0, -30.0)
(563086.4981809029, 30.0, 0.0, 4530140.279354534, 0.0, -30.0)
(563086.4981809029, 30.0, 0.0, 4530140.279354534, 0.0, -30.0)
(563086.4981809029, 30.0, 0.0, 4530140.279354534, 0.0, -30.0)
(563086.4981809029, 30.0, 0.0, 4530140.279354534, 0.0, -30.0)
(563086.4981809029, 30.0, 0.0, 4530140.27